# Let's Build an LLM-as-a-Judge Tool

In [ ]:
from pydantic import BaseModel
import polars as pl

In [ ]:
JUDGE_SYSTEM_MESSAGE = """You are a movie expert chatbot evaluator with advanced capabilities to understand if the question response is good or not.
Follow some guidelines to evaluate the system:
{guidelines}

We provide you with some examples to give you an idea of how a good response should be.
Evaluation examples:
{examples}
"""

EXAMPLE_STRUCTURE = """<user_question>{question}</user_question>
<response>{response}</response>
<outcome>{outcome}</outcome>
<critique>{critique}</critique>"""

In [ ]:
class EvaluationExample(BaseModel): 
    question: str
    response: str
    outcome: str
    critique: str

In [ ]:
evaluation_guidelines = [
    "Provide a \"pass\" or \"fail\" outcome. No other category is admitted."
    "Provide a comprehensive and clear explanation of why you gave that outcome. In particular for the \"fail\" one"
]

examples = [
    EvaluationExample(
        question="Could you suggest an action movie?",
        response="I recommend \"The Thousand Faces of Dunjia\" (2017), directed by Yuen Woo-ping. The film follows a group of swordsmen's adventures as they secretly protect humanity by hunting down mysterious creatures from outer space. It combines action and fantasy elements, making it an engaging watch for fans of the genre. Enjoy!",
        outcome="pass",
        critique="The system replied with an action movie correctly. The response is brief but well explained."
    ),
    EvaluationExample(
        question="Give me a summary of DUNE II",
        response="I'm sorry, but I don't have information about \"Dune II.\" My expertise covers movie plots, metadata, and summaries of films, but it seems that title isn't available in my current context. If you have any questions about other movies, feel free to ask!",
        outcome="fail",
        critique="The system should reply to questions regarding movie summaries. It seems that can't find the movie in the context."
    )
]

In [ ]:
def build_evaluation_examples(example: EvaluationExample) -> str:
    return EXAMPLE_STRUCTURE.format(question = example.question, 
                                    response = example.response, 
                                    outcome = example.outcome, 
                                    critique = example.critique)

def build_evaluation_system_message(system_message: str, guidelines: str, 
                                    examples: list[EvaluationExample]) -> str:
    formatted_guidelines = "\n".join(guidelines)
    formatted_examples = "\n\n".join([build_evaluation_examples(ex) for ex in examples])
    return system_message.format(guidelines = formatted_guidelines, examples = formatted_examples)

In [ ]:
formatted_system_message = build_evaluation_system_message(JUDGE_SYSTEM_MESSAGE, evaluation_guidelines, examples)
print(formatted_system_message)

In [ ]:
domain_expert_critiques = pl.read_excel(
    source="../data/eval_questions_with_critiques.xlsx",
    sheet_name="Sheet1",
)

In [ ]:
domain_expert_critiques

In [ ]:
formatted_questions = domain_expert_critiques.select(pl.format("Question: {}\nAnswer: {}\n\n", 
                                         pl.col("question"), 
                                         pl.col("rag_answer")
                                        )
                              ).to_series(0).str.join("\n").item()

In [ ]:
prompt = f"""Evaluate how our AI system answered the given questions.
Here is the list of question and answer couples: 
{formatted_questions}"""

print(prompt)

In [ ]:
class LLMJudgeDataset(BaseModel):
    critiques: list[EvaluationExample]

In [ ]:
import openai


client = openai.OpenAI()


chat_completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": formatted_system_message},
        {"role": "user", "content": prompt},
    ],
    response_format=LLMJudgeDataset
)

llm_judge_outcome = chat_completion.choices[0].message.parsed

In [ ]:
llm_judge_df = pl.from_dicts(llm_judge_outcome.model_dump()['critiques'])
llm_judge_df = llm_judge_df.rename({"outcome": "Judgement_llm_judge", "critique": "Critique_llm_judge"})
llm_judge_df

In [ ]:
full_judges = pl.concat([domain_expert_critiques, 
                         llm_judge_df.select(["Judgement_llm_judge", "Critique_llm_judge"])], 
                        how="horizontal")

In [ ]:
full_judges